# export tiles for contextal layers


In [1]:
import ee
import geemap

/opt/conda/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# Trigger the authentication flow.
# ee.ServiceAccountCredentials(EE_ACCOUNT, EE_PRIVATE_KEY_FILE)
ee.Authenticate(auth_mode='notebook')


Successfully saved authorization token.


In [2]:
# Initialize the library.
ee.Initialize()

In [3]:
def exportMapTask(asset: ee.Image, asset_name: str, gcbucket: str,
                region: ee.Geometry, min_zoom: int, max_zoom: int, 
                env: str = 'staging', key: str = 'year'):
    """
    Export the data to GEE.

    Parameters
    ----------
    asset : ee.Image
        The image to export.
    asset_name : str
        The name of the asset.
    gcbucket : str
        The name of the GCS bucket.
    geometry_collection : ee.GeometryCollection
        The geometry collection to export the data.
    
    Returns
    -------
    A task
    """
    
    
    return ee.batch.Export.map.toCloudStorage(
            image = asset,
            description = f'{asset_name}',
            path = f'{env}/tilesets/{asset_name}',
            bucket = gcbucket,
            minZoom = min_zoom,
            maxZoom = max_zoom,
            writePublicTiles = True,
            skipEmptyTiles = True,
            fileFormat = 'png',
            region = region
            )

### Data Layer: global_tidal_wetland_change 
#### Visualize the data prior exporting it

In [7]:
## Assets selection
gcs_bucket = 'mangrove_atlas'

dataset = f'global_tidal_wetland_change'
# Region to export
region = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], None, False);
# layer to export
# expresion to generate one image with the gain and loss
bandIndexExp = "b('loss') + b('gain')*2"
image = ee.Image("JCU/Murray/GIC/global_tidal_wetland_change/2019").select(['loss', 'gain']).unmask().expression(bandIndexExp).rename('gain_loss')
# masking the image
to_vis = image.mask(image.neq(0))

# exagerated image overviews for zoomed out views
overviews = to_vis.resample('bilinear').unmask().focalMode(1, 'circle', 'pixels', 2).selfMask()
# visualization parameters
visoptions =  {"palette": ['B7E6A5', 'ca5268'], "min": 1, "max": 2}


Map = geemap.Map(center=( 1.3, 103.7), zoom=12, basemap='CartoDB.PositronNoLabels')

Map.addLayer(to_vis, visoptions, f"{dataset}", True, 1)
Map.addLayer(overviews, visoptions, f"{dataset}_overviews", True, 1)

Map

Map(center=[1.3, 103.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

In [9]:
dataset_extent = f'mangrove_extent'
# Region to export
region = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], None, False);
# layer to export
# expresion to generate one image with the gain and loss
image_extent = ee.Image("projects/global-mangrove-watch/land-cover/mangrove_extent-v3/gmw_v3_2020")

# exagerated image overviews for zoomed out views
overviews_extent = image_extent.resample('bilinear').unmask().focalMode(1, 'circle', 'pixels', 3).selfMask()
# visualization parameters
visoptions =  {"palette": ['B7E6A5', 'ca5268'], "min": 1, "max": 2}

visoptions_extent =  {"palette": ['00BDB5'], "min": 1, "max": 1}

Map = geemap.Map(center=( 1.3, 103.7), zoom=12, basemap='CartoDB.PositronNoLabels')

Map.addLayer(image_extent, visoptions_extent, f"{dataset_extent}", True, 1)
Map.addLayer(overviews_extent, visoptions_extent, f"{dataset_extent}_overviews", True, 1)

Map.addLayer(to_vis, visoptions, f"{dataset}", True, 0.6)
Map.addLayer(overviews, visoptions, f"{dataset}_overviews", True, 0.6)

Map

Map(center=[1.3, 103.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

#### Export to tiles from GEE to GCS

In [10]:
task = exportMapTask(overviews.visualize(**visoptions), dataset, gcs_bucket, 
                     region, 1, 6)
# Create a list of tasks to export the map tiles
task2 = exportMapTask(to_vis.visualize(**visoptions), dataset, gcs_bucket, 
                     region, 6, 10)
# execute tasks in batches due to gee limits
task.start()
task2.start()


EEException: Bucket "mangrove_atlas" must have fine-grained ACLs. See: https://developers.google.com/storage/docs/accesscontrol#defaultbuckets

### Data Layer: Global intertidal wetland change
#### Visualize the data prior exporting it

In [7]:
## Assets selection
gcs_bucket = 'mangrove_atlas'

dataset = f'global_tidal_wetland_change'
# Region to export
region = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], None, False);
# layer to export
# expresion to generate one image with the gain and loss
bandIndexExp = "b('loss') + b('gain')*2"
image = ee.Image("JCU/Murray/GIC/global_tidal_wetland_change/2019").select(['loss', 'gain']).unmask().expression(bandIndexExp).rename('gain_loss')
# masking the image
to_vis = image.mask(image.neq(0))

# exagerated image overviews for zoomed out views
overviews = to_vis.resample('bilinear').unmask().focalMode(3, 'circle', 'pixels', 3).selfMask()
# visualization parameters
visoptions =  {"palette": ['FE4A49', 'ca5268'], "min": 1, "max": 2}


Map = geemap.Map(center=( 1.3, 103.7), zoom=12, basemap='HYBRID')

Map.addLayer(to_vis, visoptions, f"{dataset}", True, 1)
Map.addLayer(overviews, visoptions, f"{dataset}", True, 1)


#### Export to tiles from GEE to GCS

In [ ]:
task = exportMapTasks(overviews.visualize(**visoptions), f"{dataset}_overviews", gcs_bucket, 
                     region, 1, 6)
# Create a list of tasks to export the map tiles
task2 = exportMapTasks(to_vis.visualize(**visoptions), dataset, gcs_bucket, 
                     region, 6, 10)
# execute tasks in batches due to gee limits
task.start()
task2.start()


### Data Layer: Tidal flats
#### Visualize the data prior exporting it

In [5]:
## Assets selection
gcs_bucket = 'mangrove_atlas'

dataset = f'tidal_flats_1_2'
# Region to export
region = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], None, False);
# layer to export
# expresion to generate one image with the gain and loss
image = ee.Image("projects/UQ_intertidal/global_intertidal_v1_2/L5_final_masked/global_intertidal_20172019_v1_2")

# exagerated image overviews for zoomed out views
overviews = image.resample('bilinear').unmask().focalMode(1, 'circle', 'pixels', 3).selfMask()
# visualization parameters
visoptions =  {"palette": ['C1766F'], "min": 1, "max": 1}

Map = geemap.Map(center=( 1.3, 103.7), zoom=5, basemap='CartoDB.PositronNoLabels')

Map.addLayer(image, visoptions, f"{dataset}", True, 1)
Map.addLayer(overviews, visoptions, f"{dataset}_overviews", True, 1)
Map

Map(center=[1.3, 103.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

In [10]:
dataset_extent = f'mangrove_extent'
# Region to export
region = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], None, False);
# layer to export
# expresion to generate one image with the gain and loss
image_extent = ee.Image("projects/global-mangrove-watch/land-cover/mangrove_extent-v3/gmw_v3_2020")

# exagerated image overviews for zoomed out views
overviews_extent = image_extent.resample('bilinear').unmask().focalMode(1, 'circle', 'pixels', 3).selfMask()
# visualization parameters
visoptions =  {"palette": ['C1766F'], "min": 1, "max": 1}

visoptions_extent =  {"palette": ['00BDB5'], "min": 1, "max": 1}

Map = geemap.Map(center=( 1.3, 103.7), zoom=12, basemap='SATELLITE')

Map.addLayer(image, visoptions, f"{dataset}", True, 1)
Map.addLayer(overviews, visoptions, f"{dataset}_overviews", True, 1)

Map.addLayer(image_extent, visoptions_extent, f"{dataset_extent}", True, 1)
Map.addLayer(overviews_extent, visoptions_extent, f"{dataset_extent}_overviews", True, 1)



Map

Map(center=[1.3, 103.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

#### Export to tiles from GEE to GCS

In [ ]:
task = exportMapTask(overviews.visualize(**visoptions), f"{dataset}_overviews", gcs_bucket, 
                     region, 1, 6)
# Create a list of tasks to export the map tiles
task2 = exportMapTask(image.visualize(**visoptions), dataset, gcs_bucket, 
                     region, 6, 10)
# execute tasks in batches due to gee limits
task.start()
task2.start()

### Data Layer: Allen coral Atlas
#### Visualize the data prior exporting it

In [5]:
## Assets selection
gcs_bucket = 'mangrove_atlas'

dataset = f'allen_coral_atlas_v2'
# Region to export
region = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], None, False);
# layer to export

image = ee.Image("ACA/reef_habitat/v2_0").select("reef_mask")

# exagerated image overviews for zoomed out views
overviews = image.resample('bilinear').unmask().focalMode(1, 'circle', 'pixels', 3).selfMask()
# visualization parameters
visoptions =  {"palette": [ 'FF6577'], "min": 1, "max": 1}


Map = geemap.Map(center=( 1.3, 103.7), zoom=12, basemap='CartoDB.PositronNoLabels')

Map.addLayer(image, visoptions, f"{dataset}", True, 1)
Map.addLayer(overviews, visoptions, f"{dataset}_overviews", True, 1)
Map

Map(center=[1.3, 103.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

#### Export to tiles from GEE to GCS

In [6]:
task = exportMapTask(overviews.visualize(**visoptions), f"{dataset}_overviews", gcs_bucket, 
                     region, 1, 6)
# Create a list of tasks to export the map tiles
task2 = exportMapTask(image.visualize(**visoptions), dataset, gcs_bucket, 
                     region, 6, 10)
# execute tasks in batches due to gee limits
task.start()
task2.start()


EEException: Bucket "mangrove_atlas" must have fine-grained ACLs. See: https://developers.google.com/storage/docs/accesscontrol#defaultbuckets